In [ ]:
# Import library modules
import sys
import cv2 # OpenCV library
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image # PIL is the Python Imaging Library

In [ ]:
def compute_homography(src, dst):
  '''Computes the homography from src to dst.
   Input:
    src: source points, shape (N, 2), where N >= 4
    dst: destination points, shape (N, 2)
   Output:
    H: homography from source points to destination points, shape (3, 3)
  '''
  assert src.shape == dst.shape, "Input matrices must have the same shape"
  assert src.shape[0] >= 4, "At least 4 corresponding points are required"

  N = src.shape[0]

  A = np.zeros((2 * N, 9))

  for i in range(N):
    x, y = src[i, 0], src[i, 1]
    u, v = dst[i, 0], dst[i, 1]

    A[2 * i] = [-x, -y, -1, 0, 0, 0, x * u, y * u, u]
    A[2 * i + 1] = [0, 0, 0, -x, -y, -1, x * v, y * v, v]

  _, _, Vt = np.linalg.svd(A)
  H = Vt[-1].reshape(3, 3)
  H /= H[2, 2]

  return H

###############################################
# Implement the apply_homography function

def apply_homography(src, H):
  '''Applies a homography H to the source points.
   Input:
      src: source points, shape (N, 2)
      H: homography from source points to destination points, shape (3, 3)
   Output:
     dst: destination points, shape (N, 2)
  '''
  assert src.shape[1] == 2, "src must have two columns (x, y)"
  assert H.shape == (3, 3), "H' must be a 3x3 matrix"

  N = src.shape[0]

  src_homogeneous = np.hstack((src, np.ones((N, 1))))
  dst_homogeneous = np.dot(src_homogeneous, H.T)
  dst = dst_homogeneous[:, :2] / dst_homogeneous[:, 2].reshape(-1, 1)

  return dst

Test: so here the output (difference)  should be less than 0

In [ ]:
def test_homography():
  src_pts = np.matrix('0, 0; 1, 0; 1, 1; 0, 1')
  dst_pts = np.matrix('3, 2; 3.67, 2; 3.5, 2.5; 3, 3')
  H = compute_homography(src_pts, dst_pts)
  test_pts = src_pts
  match_pts = apply_homography(test_pts, H)
  match_pts_correct = dst_pts
  print('Your 1st solution differs from our solution by: %f'
    % np.square(match_pts - match_pts_correct).sum())

  src_pts = np.matrix('0, 0; 1, 0; 1, 1; 0, 1; 2, 3')
  dst_pts = np.matrix('3, 2; 3.67, 2; 3.5, 2.5; 3, 3; 3.5, 2.75')
  H = compute_homography(src_pts, dst_pts)
  test_pts = np.matrix('0,  0; 1, 0; 1, 1; 0, 1')
  match_pts = apply_homography(test_pts, H)
  match_pts_correct = np.matrix('3, 2; 3.67, 2; 3.5, 2.5; 3, 3')
  print('Your 2nd solution differs from our solution by: %f'
    % np.square(match_pts - match_pts_correct).sum())

  src_pts = np.matrix('347, 313; 502, 341; 386, 571; 621, 508')
  dst_pts = np.matrix('274, 286; 436, 305; 305, 527; 615, 506')
  H = compute_homography(src_pts, dst_pts)
  test_pts = np.matrix('259, 505; 350, 371; 400, 675; 636, 104')
  match_pts = apply_homography(test_pts, H)
  match_pts_correct = np.matrix('195.13761083, 448.12645033;'
    '275.27269386, 336.54819916;'
    '317.37663747, 636.78403426;'
    '618.50438823, 28.78963905')
  print('Your 3rd solution differs from our solution by: %f'
    % np.square(match_pts - match_pts_correct).sum())

test_homography()

Image wrapping function

In [ ]:

def warp_img(src_img, H, dst_img_size):
    # Create an empty destination image
    dst_img = np.zeros((dst_img_size[0], dst_img_size[1], 3))
    # Calculate the inverse of the homography matrix
    H_inv = np.linalg.inv(H)

    # Iterate over all pixels in dst_img
    for i in range(dst_img_size[0]):
      for j in range(dst_img_size[1]):
        point = np.array([[i], [j], [1]])
        transformed_point = np.dot(H_inv, point)

        src_x = int(np.round(transformed_point[0] / transformed_point[2]))
        src_y = int(np.round(transformed_point[1] / transformed_point[2]))

        if 0 <= src_x < src_img.shape[0] and 0 <= src_y < src_img.shape[1]:
           dst_img[i, j, :] = src_img[src_x, src_y, :]

    return dst_img

In [ ]:
def binary_mask(img):
  '''Create a binary mask of the image content.
   Input:
    img: source image, shape (m, n, 3)
   Output:
    mask: image of shape (m, n) and type 'int'. For pixel [i, j] of mask,
      if pixel [i, j] in img is nonzero in any channel, assign 1 to mask[i, j].
      Else, assign 0 to mask[i, j].
  '''
  mask = (img[:, :, 0] > 0) | (img[:, :, 1] > 0) | (img[:, :, 2] > 0)
  mask = mask.astype("int")
  return mask

def test_warp():
  src_img = load_image('mandrill.png')
  canvas_img = load_image('Rubiks_cube.jpg')

  # The following are corners of the mandrill image in (ROW, COLUMN) order
  src_pts = np.matrix('0, 0; 511, 0; 511, 511; 0, 511')
  # The following are corners of the blue face of the Rubik's cube
  canvas_pts = np.matrix('238, 218; 560, 225; 463, 490; 178, 530')

  # The following was used during debugging
  # It draws a circle at a location specified by (COLUMN, ROW)
  # cv2.circle(canvas_img, (530, 178), 4, (255, 0, 0), thickness=10)

  H = compute_homography(src_pts, canvas_pts)
  dst_img = warp_img(src_img, H, [canvas_img.shape[0], canvas_img.shape[1]])
  dst_mask = 1 - binary_mask(dst_img)
  dst_mask = np.stack((dst_mask,) * 3, -1)
  out_img = np.multiply(canvas_img, dst_mask) + dst_img

  dsize = (600, 600) # width and height of canvas_im
  src_smaller = cv2.resize(src_img, dsize, interpolation=cv2.INTER_AREA)

  warped_img = np.concatenate((src_smaller, canvas_img, out_img), axis=1)
  show_image(np.clip(warped_img, 0, 1))

test_warp()